# Get the teradata-mcp-server (Community Edition) Code

In [ ]:
import os
if not os.path.exists("teradata-mcp-server"):
    !git clone  "https://github.com/Teradata/teradata-mcp-server.git"
else:
    !git -C "teradata-mcp-server" pull

In [ ]:
import tomllib,shlex

with open("./teradata-mcp-server/pyproject.toml", "rb") as f:
    deps = tomllib.load(f)["project"]["dependencies"]
deps_str = " ".join(shlex.quote(d) for d in deps)

print(deps_str)

In [ ]:
%%capture
# # '%%capture' suppresses the display of installation steps of the following packages
%pip install -U {deps_str}

# Spin up Teradata MCP

## Set db credentials

for the MCP servers to talk to Teradata, we need to give them the db credentials

In [ ]:
host = 'host.docker.internal'
user = 'demo_user'

In [ ]:
import getpass
db_pw = getpass.getpass("Enter DB Password here")

## start the server

In [ ]:
import os
import chatbot_widget
from chatbot_widget.mcp.server_manager import MCPServerManager

mcp = MCPServerManager()

In [ ]:
mcp.inspect_cli_arguments(script_path="tdmcpcommunity.py")

In [ ]:
mcp.start(
    "tdmcpcommunity",
    os.path.abspath("tdmcpcommunity.py"),
    port=None,  # or fixed port like 8765
    host=host,
    user=user,
    password=db_pw
)

In [ ]:
import time
#check after 5 seconds, DB connection needs to start
time.sleep(5)
mcp.check_health("tdmcpcommunity")

In [ ]:
mcp.list_tools("tdmcpcommunity")

In [ ]:
response = mcp.test_tool("tdmcpcommunity","dba_databaseVersion")

In [ ]:
# structure of response is prepared for LLM consumption
print(response)

In [ ]:
import json
json.loads(response['result'].content[0].text)['results']

# chatbot

In [ ]:
import os
import getpass

In [ ]:
open_api_key = getpass.getpass("Enter Open API Key")
os.environ["OPENAI_API_KEY"] = open_api_key

In [ ]:
from chatbot_widget.controller.chat_mcp_controller import ChatMCPController
# you can ignore the warnings
mychat = ChatMCPController(mcp, "openai:gpt-4o-mini")
#mychat = ChatMCPController(mcp, "google_genai:gemini-2.5-flash")

***Some notes:**

- note this is not a product grade chat interface; It's a purely exploratory tool
- start with commands like /tools to see what tools the Agent can access.
- start witha simple query like, `what tables are in my demo_user db?`
- you can also ask complex questions like 
    - `are there data quality issues with the payments?` 
    or 
    - `give me a markdown formatted business summary of the data in table, like whats it about and what analytics use cases you could do with it. it should be executive level, less then 300 words in total.`
- be patient. Currently, the interface outputs everything at once.

In [ ]:
mychat.display()

### Stop the MCP Sever

In [21]:
mcp.stop_all()

[{'status': 'ok',
  'message': "Stopped 'tdmcpcommunity' (port=8887, PID=1288)."}]